In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import linregress
from itertools import product
import math

pd.options.plotting.backend = "plotly"

In [ ]:
# https://docs.screeps.com/api/#Creep
CREEP_TICKS_LIFE = 1500
MAX_BODY_SIZE = 50
MAX_DISTANCE = 50

In [ ]:
# https://docs.screeps.com/creeps.html#Movement'

class FixedWalk:
    """Assuming one moving part for each carry part"""

    @classmethod
    def harvested_tick(cls, work):
        return work * 2

    @classmethod
    def capacity(cls, carry):
        return carry * 50

    @classmethod
    def ticks_to_full(cls, work, carry):
        return cls.capacity(carry) / cls.harvested_tick(work)

    @classmethod
    def round_trip(cls, work, carry, distance):
        return (distance * 2) + cls.ticks_to_full(work, carry) + 1

    @classmethod
    def body_cost(cls, work, carry):
        move = carry + work
        return work * 100 + carry * 50 + move * 50

    @classmethod
    def profit(cls, work, carry, distance):
        return round(CREEP_TICKS_LIFE / cls.round_trip(work, carry, distance) * cls.capacity(carry) - cls.body_cost(work, carry))

    @classmethod
    def cost_benefit(cls, work, carry, distance):
        return cls.profit(work, carry, distance) / cls.body_cost(work, carry)

    @classmethod
    def cost_benefit_fixed_walk(cls):
        actual_max_size = round(MAX_BODY_SIZE / 4)
        work_parts = [*range(1, actual_max_size)]
        carry_parts = [*range(1, actual_max_size)]
        distances = [*range(5, MAX_DISTANCE, 5)]

        cost_benefit_heatmap = list(map(lambda wc: [str(wc), wc[0], wc[1], wc[2], cls.cost_benefit(*wc)], product(work_parts, carry_parts, distances)))
        df = pd.DataFrame(cost_benefit_heatmap, columns=["body","work_parts", "carry_parts", "distance", "cost_benefit"])
        df["ratio"] = df['carry_parts'] / df['work_parts']
        df = df.sort_values(by="cost_benefit")
        df['ratio'] = round(df['ratio'], 4)
        df['cost_benefit'] = round(df['cost_benefit'], 4)

        best_by_distance = df.groupby(['distance']).agg({'cost_benefit': 'max'})
        print(best_by_distance)
        print("")

        df_reg = df[df['cost_benefit'].isin(best_by_distance['cost_benefit'])]
        df_reg = df_reg.sort_values('distance').copy()
        print(df_reg)
        print("")

        regression = linregress(df_reg['distance'], df_reg['ratio'])
        print(regression)
        print("")

        print(round(regression.slope, 3), round(regression.intercept, 3))
        print("")

        return px.scatter(df, x="ratio", y="cost_benefit", color="distance")

FixedWalk.cost_benefit_fixed_walk()

In [28]:
class Proportions:
    """Not assuming one moving part for each carry part"""

    @classmethod
    def harvested_tick(cls, work):
        return work * 2

    @classmethod
    def capacity(cls, carry):
        return carry * 50

    @classmethod
    def ticks_to_full(cls, work, carry):
        return cls.capacity(carry) / cls.harvested_tick(work)

    @classmethod
    def ticks_per_move(cls, work, carry, move, direction="Forward"):
        return math.floor(work / move) if direction == "Forward" else math.floor((work + carry) / move)

    @classmethod
    def round_trip(cls, work, carry, move, distance):
        forward = distance * cls.ticks_per_move(work, carry, move, "Forward") + distance
        backwards = distance * cls.ticks_per_move(work, carry, move, "Backwards") + distance
        return (forward + backwards) + cls.ticks_to_full(work, carry) + 1

    @classmethod
    def body_cost(cls, work, carry, move):
        return work * 100 + carry * 50 + move * 50

    @classmethod
    def profit(cls, work, carry, move, distance):
        round_trips = round(CREEP_TICKS_LIFE / cls.round_trip(work, carry, distance, move))
        return round_trips * cls.capacity(carry) - cls.body_cost(work, carry, move)

    @classmethod
    def cost_benefit(cls, work, carry, move, distance):
        return cls.profit(work, carry, distance, move) / cls.body_cost(work, carry, move)
    
    @classmethod
    def cost_benefit_regression(cls):
        distances = [*range(5, MAX_DISTANCE, 5)]

        work_parts = [*range(1, MAX_BODY_SIZE)]
        carry_parts = [*range(1, MAX_BODY_SIZE)]
        move_parts = [*range(1, MAX_BODY_SIZE)]

        # Body combinations with less than MAX_BODY_SIZE parts
        parts_prod = filter(lambda x: (x[0] + x[1] + x[2]) < MAX_BODY_SIZE, product(work_parts, carry_parts, move_parts))
        parts_prod = filter(lambda x: cls.ticks_per_move(*x) < 10, parts_prod)
        # parts_prod

        cost_benefit_heatmap = list(map(lambda wc: [str(wc), *wc[0], wc[1], cls.cost_benefit(*wc[0], wc[1])], product(parts_prod, distances)))
        df = pd.DataFrame(cost_benefit_heatmap, columns=["label", "work_parts", "carry_parts", "walk_parts", "distance", "cost_benefit"])
        df = df[df['cost_benefit']>1]
        df['cost_benefit'] = round(df['cost_benefit'], 4)
        return df.copy()

df = Proportions.cost_benefit_regression()
df

,label,work_parts,carry_parts,walk_parts,distance,cost_benefit
0,"((1, 1, 1), 5)",1,1,1,5,5.2500
1,"((1, 1, 1), 10)",1,1,1,10,1.7500
9,"((1, 1, 2), 5)",1,1,2,5,5.8000
10,"((1, 1, 2), 10)",1,1,2,10,2.8000
18,"((1, 1, 3), 5)",1,1,3,5,5.6667
...,...,...,...,...,...,...
152433,"((34, 7, 7), 5)",34,7,7,5,1.1585
152442,"((34, 7, 8), 5)",34,7,8,5,1.1446
152478,"((34, 8, 7), 5)",34,8,7,5,1.1446
152505,"((34, 9, 6), 5)",34,9,6,5,1.0723


In [29]:
df.groupby(['distance']).agg({'cost_benefit': 'max'})

,cost_benefit
distance,
5,8.2923
10,6.0714
15,4.8136
20,3.9286
25,3.2632
30,2.8421
35,2.3519
40,2.0714
45,1.7368


In [37]:
best_by_distance = df.groupby(['distance']).agg({'cost_benefit': 'max'})
best_by_distance = best_by_distance.reset_index()
best_by_distance['q'] = best_by_distance['distance'].astype(str) + best_by_distance['cost_benefit'].astype(str)
print(best_by_distance)
print("")


df['q'] = df['distance'].astype(str) + df['cost_benefit'].astype(str)
df_reg = df[df['q'].isin(best_by_distance['q'])]
df_reg = df_reg.sort_values(['distance', 'cost_benefit'], ascending=False).copy()
df_reg

   distance  cost_benefit         q
0         5        8.2923   58.2923
1        10        6.0714  106.0714
2        15        4.8136  154.8136
3        20        3.9286  203.9286
4        25        3.2632  253.2632
5        30        2.8421  302.8421
6        35        2.3519  352.3519
7        40        2.0714  402.0714
8        45        1.7368  451.7368



,label,work_parts,carry_parts,walk_parts,distance,cost_benefit
67175,"((8, 16, 25), 45)",8,16,25,45,1.7368
59794,"((7, 17, 25), 40)",7,17,25,40,2.0714
59550,"((7, 16, 24), 35)",7,16,24,35,2.3519
59334,"((7, 15, 27), 35)",7,15,27,35,2.0714
73716,"((9, 13, 25), 35)",9,13,25,35,2.0714
...,...,...,...,...,...,...
103014,"((14, 12, 2), 5)",14,12,2,5,2.0714
105651,"((15, 2, 10), 5)",15,2,10,5,2.0714
17676,"((2, 25, 9), 5)",2,25,9,5,1.7368
20151,"((3, 1, 31), 5)",3,1,31,5,1.7368
